In [1]:
#NN library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import torch
from typing import Optional, Union, Tuple


import torch.nn.functional as F
from torch.distributions import Distribution
from torch.distributions import Bernoulli, Normal, StudentT, Poisson, NegativeBinomial

from torch.distributions import constraints

from ray import tune

from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATSx, NHITS
from neuralforecast.auto import AutoNHITS, AutoLSTM
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from neuralforecast.losses.pytorch import MSE

In [2]:
from numpy.random import seed
from random import randrange

In [3]:
#Realized volatility 
def Yang_Zhang_RV_yahoo(tickers, start=None, end=None, period=None, interval=None):
    # importing needed libraries
    import yfinance as yf
    import pandas as pd
    import numpy as np
    import warnings
    warnings.filterwarnings("ignore")
    
    #Data extraction
    if period==None:
        data=yf.download(tickers=tickers, start=start, end=end, interval=interval)
    else:
        data=yf.download(tickers=tickers, period=period, interval=interval)
    #dropping N/A values
    if data.isnull().values.any()==True:
        data=data.dropna()
        print("Rows with missing values were removed")
    else:
        data=data
        
    # Yang_Zhang_RV formula is give as:
    # RV^2 = Vo + k*Vc + (1-k)*Vrs
    # where Vo = 1/(n-1)*sum(Oi-Obar)^2
    # with oi = normalized opening price at time t and Obar = mean of normalized opening prices
    # Vc = = 1/(n-1)*sum(ci-Cbar)^2
    # with ci = normalized close price at time t and Cbar = mean of normalized close prices
    # k = 0.34/(1.34+(n+1)/(n-1))
    # with n = total number of days or time periods considered
    # Vrs (Rogers & Satchell RV proxy) = ui(ui-ci)+di(di-ci)
    # with ui = ln(Hi/Oi), ci = ln(Ci/Oi), di=(Li/Oi), oi = ln(Oi/Ci-1)
    # where Hi = high price at time t and Li = low price at time t
    
    data["ui"]=np.log(np.divide(data["High"][1:],data["Open"][1:]))
    data["ci"]=np.log(np.divide(data["Close"][1:],data["Open"][1:]))
    data["di"]=np.log(np.divide(data["Low"][1:],data["Open"][1:]))
    data["oi"]=np.log(np.divide(data["Open"][1:],data["Close"][:len(data)-1].shift(1)))
    data=data[1:]
    data["RS"]=data["ui"]*(data["ui"]-data["ci"])+data["di"]*(data["di"]-data["ci"])
    RS_var= data["RS"].groupby(pd.Grouper(freq='W')).mean().dropna()
    Vc_and_Vo=data[["oi", "ci"]].groupby(pd.Grouper(freq='W')).var().dropna()
    n=int(len(data)/len(RS_var))
    k = 0.34/(1.34+(n+1)/(n-1))
    Yang_Zhang_RV=np.sqrt((1-k)*RS_var+Vc_and_Vo["oi"]+Vc_and_Vo["ci"]*k)
    Yang_Zhang_RV_df=pd.DataFrame(Yang_Zhang_RV)
    Yang_Zhang_RV_df.rename(columns={0: "Yang & Zhang RV proxy"},inplace=True)
    
    return Yang_Zhang_RV_df

def Yang_Zhang_RV_own_data(data):
    # importing needed libraries
    import pandas as pd
    import numpy as np
    import warnings
    warnings.filterwarnings("ignore")
    
    # Yang_Zhang_RV formula is give as:
    # RV^2 = Vo + k*Vc + (1-k)*Vrs
    # where Vo = 1/(n-1)*sum(Oi-Obar)^2
    # with oi = normalized opening price at time t and Obar = mean of normalized opening prices
    # Vc = = 1/(n-1)*sum(ci-Cbar)^2
    # with ci = normalized close price at time t and Cbar = mean of normalized close prices
    # k = 0.34/(1.34+(n+1)/(n-1))
    # with n = total number of days or time periods considered
    # Vrs (Rogers & Satchell RV proxy) = ui(ui-ci)+di(di-ci)
    # with ui = ln(Hi/Oi), ci = ln(Ci/Oi), di=(Li/Oi), oi = ln(Oi/Ci-1)
    # where Hi = high price at time t and Li = low price at time t
    
    data["ui"]=np.log(np.divide(data["High"][1:],data["Open"][1:]))
    data["ci"]=np.log(np.divide(data["Close"][1:],data["Open"][1:]))
    data["di"]=np.log(np.divide(data["Low"][1:],data["Open"][1:]))
    data["oi"]=np.log(np.divide(data["Open"][1:],data["Close"][:len(data)-1].shift(1)))
    data=data[1:]
    data["RS"]=data["ui"]*(data["ui"]-data["ci"])+data["di"]*(data["di"]-data["ci"])
    RS_var= data["RS"].groupby(pd.Grouper(freq='D')).mean().dropna()
    Vc_and_Vo=data[["oi", "ci"]].groupby(pd.Grouper(freq='D')).var().dropna()
    n=int(len(data)/len(RS_var))
    k = 0.34/(1.34+(n+1)/(n-1))
    Yang_Zhang_RV=np.sqrt((1-k)*RS_var+Vc_and_Vo["oi"]+Vc_and_Vo["ci"]*k)
    Yang_Zhang_RV_df=pd.DataFrame(Yang_Zhang_RV)
    Yang_Zhang_RV_df.rename(columns={0: "Yang & Zhang RV proxy"},inplace=True)
    
    return Yang_Zhang_RV_df
    
    
    
def Multivariate_Yang_Zhang_RV_own_data(data_list):
    Multivariate_Yang_Zhang_RV=[]
    for i in range(len(data_list)):
        Yang_Zhang_RV_df=Yang_Zhang_RV_own_data(data=data_list[i])
        Multivariate_Yang_Zhang_RV.append(Yang_Zhang_RV_df)
    return Multivariate_Yang_Zhang_RV
    
    
def Multivariate_Yang_Zhang_RV_yahoo(tickers, start=None, end=None, period=None, interval=None):
    # importing needed libraries
    import yfinance as yf
    import pandas as pd
    import numpy as np
    import warnings
    warnings.filterwarnings("ignore")
    
    #Data extraction
    if period==None:
        data=yf.download(tickers=tickers, start=start, end=end, interval=interval)
    else:
        data=yf.download(tickers=tickers, period=period, interval=interval)
    #dropping N/A values
    if data.isnull().values.any()==True:
        data=data.dropna()
        print("Rows with missing values were removed")
    else:
        data=data

    data=data.unstack().reset_index(name="Actuals").rename(columns={"level_1":"Stocks"}).set_index("Datetime").pivot(columns=['Stocks','level_0'])
    data=data['Actuals']
        
    # Yang_Zhang_RV formula is give as:
    # RV^2 = Vo + k*Vc + (1-k)*Vrs
    # where Vo = 1/(n-1)*sum(Oi-Obar)^2
    # with oi = normalized opening price at time t and Obar = mean of normalized opening prices
    # Vc = = 1/(n-1)*sum(ci-Cbar)^2
    # with ci = normalized close price at time t and Cbar = mean of normalized close prices
    # k = 0.34/(1.34+(n+1)/(n-1))
    # with n = total number of days or time periods considered
    # Vrs (Rogers & Satchell RV proxy) = ui(ui-ci)+di(di-ci)
    # with ui = ln(Hi/Oi), ci = ln(Ci/Oi), di=(Li/Oi), oi = ln(Oi/Ci-1)
    # where Hi = high price at time t and Li = low price at time t
    
    Multivariate_Yang_Zhang_RV=[]
    for i in range(len(tickers)):
        data1=data[tickers[i]]
        data1["ui"]=np.log(np.divide(data1["High"][1:],data1["Open"][1:]))
        data1["ci"]=np.log(np.divide(data1["Close"][1:],data1["Open"][1:]))
        data1["di"]=np.log(np.divide(data1["Low"][1:],data1["Open"][1:]))
        data1["oi"]=np.log(np.divide(data1["Open"][1:],data1["Close"][:len(data1)-1].shift(1)))
        data1=data1[1:]
        data1["RS"]=data1["ui"]*(data1["ui"]-data1["ci"])+data1["di"]*(data1["di"]-data1["ci"])
        RS_var= data1["RS"].groupby(pd.Grouper(freq='D')).mean().dropna()
        Vc_and_Vo=data1[["oi", "ci"]].groupby(pd.Grouper(freq='D')).var().dropna()
        n=int(len(data1)/len(RS_var))
        k = 0.34/(1.34+(n+1)/(n-1))
        Yang_Zhang_RV=np.sqrt((1-k)*RS_var+Vc_and_Vo["oi"]+Vc_and_Vo["ci"]*k)
        Yang_Zhang_RV_df=pd.DataFrame(Yang_Zhang_RV)
        Yang_Zhang_RV_df.rename(columns={0: "Yang & Zhang RV proxy"},inplace=True)
        Multivariate_Yang_Zhang_RV.append(Yang_Zhang_RV_df)
    
    return Multivariate_Yang_Zhang_RV

In [4]:
import pandas as pd
from binance import Client
from dotenv import dotenv_values
from datetime import datetime

config = dotenv_values('.env')
client = Client(config.get('KEY'), config.get('SECRET_KEY'))
TICKER = 'BTCUSDT'
start_date = datetime(2018, 1, 1)
end_date = datetime(2024, 3, 1)

start_date_str = start_date.strftime('%d %b, %Y')
end_date_str = end_date.strftime('%d %b, %Y')

klines = client.get_historical_klines(TICKER, client.KLINE_INTERVAL_1HOUR, start_date_str, end_date_str)
dataBTC = pd.DataFrame(
    data=[row[1:7] for row in klines],
    columns=['Open', 'High', 'Low', 'Close', 'Volume', 'Date'],
).set_index('Date')
dataBTC.index = pd.to_datetime(dataBTC.index, unit='ms')
dataBTC = dataBTC.sort_index()
dataBTC = dataBTC.apply(pd.to_numeric, axis=1)
dataBTC

,Open,High,Low,Close,Volume
Date,,,,,
2018-01-01 00:59:59.999,13715.65,13715.65,13400.01,13529.01,443.356199
2018-01-01 01:59:59.999,13528.99,13595.89,13155.38,13203.06,383.697006
2018-01-01 02:59:59.999,13203.00,13418.43,13200.00,13330.18,429.064572
2018-01-01 03:59:59.999,13330.26,13611.27,13290.00,13410.03,420.087030
2018-01-01 04:59:59.999,13434.98,13623.29,13322.15,13601.01,340.807329
...,...,...,...,...,...
2024-02-29 20:59:59.999,61599.99,62285.47,61521.73,61934.73,3755.220100
2024-02-29 21:59:59.999,61934.73,61999.75,60584.07,61374.94,4040.139080
2024-02-29 22:59:59.999,61374.95,61474.81,60672.82,61224.02,1906.566300


In [5]:
TICKER = 'ETHUSDT'
start_date = datetime(2018, 1, 1)
end_date = datetime(2024, 3, 1)

start_date_str = start_date.strftime('%d %b, %Y')
end_date_str = end_date.strftime('%d %b, %Y')

klines = client.get_historical_klines(TICKER, client.KLINE_INTERVAL_1HOUR, start_date_str, end_date_str)
dataETH = pd.DataFrame(
    data=[row[1:7] for row in klines],
    columns=['Open', 'High', 'Low', 'Close', 'Volume', 'Date'],
).set_index('Date')
dataETH.index = pd.to_datetime(dataETH.index, unit='ms')
dataETH = dataETH.sort_index()
dataETH = dataETH.apply(pd.to_numeric, axis=1)
dataETH

,Open,High,Low,Close,Volume
Date,,,,,
2018-01-01 00:59:59.999,733.01,734.52,720.03,727.62,2105.90100
2018-01-01 01:59:59.999,727.01,732.00,716.80,717.97,2305.97086
2018-01-01 02:59:59.999,717.67,725.75,717.59,724.05,2166.45725
2018-01-01 03:59:59.999,723.95,737.99,722.70,734.50,2160.90450
2018-01-01 04:59:59.999,734.99,744.98,730.01,744.82,2335.33705
...,...,...,...,...,...
2024-02-29 20:59:59.999,3403.27,3415.44,3393.61,3393.99,18045.00550
2024-02-29 21:59:59.999,3393.99,3398.63,3310.00,3347.74,40472.54590
2024-02-29 22:59:59.999,3347.73,3358.53,3300.00,3322.61,24605.94420


In [6]:
RVBTC=Yang_Zhang_RV_own_data(dataBTC)
RVBTC['unique_id']=f'BTC'
RVBTC

,Yang & Zhang RV proxy,unique_id
Date,,
2018-01-01,0.018721,BTC
2018-01-02,0.024600,BTC
2018-01-03,0.020226,BTC
2018-01-04,0.018651,BTC
2018-01-05,0.020923,BTC
...,...,...
2024-02-26,0.008030,BTC
2024-02-27,0.010189,BTC
2024-02-28,0.018636,BTC


In [7]:
RVETH=Yang_Zhang_RV_own_data(dataETH)
RVETH['unique_id']=f'ETH'

RVETH

,Yang & Zhang RV proxy,unique_id
Date,,
2018-01-01,0.015968,ETH
2018-01-02,0.033049,ETH
2018-01-03,0.026744,ETH
2018-01-04,0.026073,ETH
2018-01-05,0.022854,ETH
...,...,...
2024-02-26,0.008802,ETH
2024-02-27,0.008446,ETH
2024-02-28,0.019383,ETH


In [8]:
static = [['BTC',1,0,], ['ETH',0,1] ]
static_df = pd.DataFrame(static, columns = ['unique_id','BTC','ETH'])
static_df

,unique_id,BTC,ETH
0,BTC,1,0
1,ETH,0,1


In [9]:
tabData=RVETH.append(RVBTC)
tabData = tabData.rename(columns={"Yang & Zhang RV proxy":"y"})
tabData['ds']=tabData.index.strftime('%Y-%m-%d')
tabData['ds']=pd.to_datetime(tabData['ds'])
tabData

,y,unique_id,ds
Date,,,
2018-01-01,0.015968,ETH,2018-01-01
2018-01-02,0.033049,ETH,2018-01-02
2018-01-03,0.026744,ETH,2018-01-03
2018-01-04,0.026073,ETH,2018-01-04
2018-01-05,0.022854,ETH,2018-01-05
...,...,...,...
2024-02-26,0.008030,BTC,2024-02-26
2024-02-27,0.010189,BTC,2024-02-27
2024-02-28,0.018636,BTC,2024-02-28


In [10]:
tabData=tabData.dropna()
tabData

,y,unique_id,ds
Date,,,
2018-01-01,0.015968,ETH,2018-01-01
2018-01-02,0.033049,ETH,2018-01-02
2018-01-03,0.026744,ETH,2018-01-03
2018-01-04,0.026073,ETH,2018-01-04
2018-01-05,0.022854,ETH,2018-01-05
...,...,...,...
2024-02-25,0.003153,BTC,2024-02-25
2024-02-26,0.008030,BTC,2024-02-26
2024-02-27,0.010189,BTC,2024-02-27


In [11]:
# 2D-Wasserstein loss function
from ripser import Rips
import persim
def _divide_no_nan(a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
    """
    Auxiliary funtion to handle divide by 0
    """
    div = a / b
    div[div != div] = 0.0
    div[div == float("inf")] = 0.0
    return div

def _weighted_mean(losses, weights):
    """
    Compute weighted mean of losses per datapoint.
    """
    return _divide_no_nan(torch.sum(losses * weights), torch.sum(weights))

# %% ../../nbs/losses.pytorch.ipynb 7
class BasePointLoss(torch.nn.Module):
    """
    Base class for point loss functions.

    **Parameters:**
    `horizon_weight`: Tensor of size h, weight for each timestamp of the forecasting window. 
    `outputsize_multiplier`: Multiplier for the output size. 
    `output_names`: Names of the outputs. 
    """

    def __init__(self, horizon_weight, outputsize_multiplier, output_names):
        super(BasePointLoss, self).__init__()
        if horizon_weight is not None:
            horizon_weight = torch.Tensor(horizon_weight.flatten())
        self.horizon_weight = horizon_weight
        self.outputsize_multiplier = outputsize_multiplier
        self.output_names = output_names
        self.is_distribution_output = False

    def domain_map(self, y_hat: torch.Tensor):
        """
        Univariate loss operates in dimension [B,T,H]/[B,H]
        This changes the network's output from [B,H,1]->[B,H]
        """
        return y_hat.squeeze(-1)

    def _compute_weights(self, y, mask):
        """
        Compute final weights for each datapoint (based on all weights and all masks)
        Set horizon_weight to a ones[H] tensor if not set.
        If set, check that it has the same length as the horizon in x.
        """
        if mask is None:
            mask = torch.ones_like(y).to(y.device)

        if self.horizon_weight is None:
            self.horizon_weight = torch.ones(mask.shape[-1])
        else:
            assert mask.shape[-1] == len(
                self.horizon_weight
            ), "horizon_weight must have same length as Y"

        weights = self.horizon_weight.clone()
        weights = torch.ones_like(mask, device=mask.device) * weights.to(mask.device)
        return weights * mask


class MSE_2DWD(BasePointLoss):
    from ripser import Rips
    import persim

    def __init__(self, horizon_weight=None):
        super(MSE_2DWD, self).__init__(
            horizon_weight=horizon_weight, outputsize_multiplier=1, output_names=[""]
        )

    def __call__(
        self,
        y: torch.Tensor,
        y_hat: torch.Tensor,
        mask: Union[torch.Tensor, None] = None,
    ):
        """
        **Parameters:**
        `y`: tensor, Actual values.
        `y_hat`: tensor, Predicted values.
        `mask`: tensor, Specifies datapoints to consider in loss.

        **Returns:**
        `mse`: tensor (single value).
        """
        rips = Rips(maxdim = 2, verbose=False)
        n=2
        wasserstein_dists = np.zeros((n))

        for i in range(2):
          dgm1 = rips.fit_transform(y_hat[:,int(y_hat.shape[1]/2*i):int(y_hat.shape[1]/2*(i+1))].to('cpu').detach().numpy())
          dgm2 = rips.fit_transform(y[:,int(y_hat.shape[1]/2*i):int(y_hat.shape[1]/2*(i+1))].to('cpu').detach().numpy())
          wasserstein_dists[i] = persim.wasserstein(dgm1[0], dgm2[0], matching=False)

        losses = torch.sum((y - y_hat) ** 2)/(y_hat.shape[0]*y_hat.shape[1]) + torch.tensor((np.sum(wasserstein_dists))/n)
        return losses
     

In [12]:
#creating train dataset
train = tabData.iloc[:int(len(RVBTC)*0.7)]
validation_length=int(len(RVBTC)*0.7)-int(len(RVBTC)*0.5)
train = tabData[tabData['ds']<= '2022-04-26']
train


,y,unique_id,ds
Date,,,
2018-01-01,0.015968,ETH,2018-01-01
2018-01-02,0.033049,ETH,2018-01-02
2018-01-03,0.026744,ETH,2018-01-03
2018-01-04,0.026073,ETH,2018-01-04
2018-01-05,0.022854,ETH,2018-01-05
...,...,...,...
2022-04-22,0.007001,BTC,2022-04-22
2022-04-23,0.004106,BTC,2022-04-23
2022-04-24,0.005758,BTC,2022-04-24


In [13]:
#Hyperparameter search

n_inputs =[3,5,10,15,21,42,84]
mlp_units = [[[712, 712], [712, 712]],[[512, 512], [512, 512]],[[250, 250], [250, 250]],[[100, 100], [100, 100]]]
epochs=[50,100,150,250,350,450,550,650,750]
learning_rate=[0.0005,0.0001,0.00005,0.00001]
num_lr_decays=[5,3,2,1]
scaler_type=["robust","standard",'minmax']
dropout_prob_theta=[0, 0.2, 0.3, 0.4]
n_pool_kernel_size=[[2,2,1], [4,2,1], [4,4,2]]
pooling_mode=['MaxPool1d', 'AvgPool1d']
interpolation_mode=['linear', 'nearest']
n_blocks=[[1, 1],[2, 2],[3, 3],[5, 5]]


In [14]:
import random
for m in range (1):
  i=random.randrange(7)
  h=random.randrange(4)
  a=random.randrange(4)
  k=random.randrange(9)
  l=random.randrange(4)
  n=random.randrange(4)
  o=random.randrange(3)
  p=random.randrange(2)
  model = NHITS(h=28, input_size=n_inputs[i],
                loss=MSE(),
                scaler_type=scaler_type[o],
                learning_rate=learning_rate[n],
                stack_types=['identity','identity'],
                n_blocks=n_blocks[a],
                mlp_units=mlp_units[h],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[n],
                val_check_steps=500,
                stat_exog_list=['BTC'],
                dropout_prob_theta=dropout_prob_theta[l], n_pool_kernel_size=n_pool_kernel_size[o],
                pooling_mode = pooling_mode[p],
                interpolation_mode= interpolation_mode[p],
                max_steps=epochs[k],
                random_seed=random.randrange(129228148))
  fcst = NeuralForecast(models=[model],freq='D')
  forecasts = fcst.cross_validation(df=train,val_size=28,static_df=static_df,n_windows=None, test_size=validation_length-validation_length%28,step_size=1)
  forecasts = forecasts[forecasts['unique_id']=='ETH'].dropna().drop_duplicates(subset='ds', keep='first')
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHTS-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | MLP units: {mlp_units[h]} | Scaler Type: {scaler_type[o]}")
  print(f"Pooling mode: {pooling_mode[p]} | Learning Rate: {n} | Block: {n_blocks[a]}| Dropout:{dropout_prob_theta[l]}  ")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAPE: {MAE}")

Seed set to 12830740
2024-03-18 16:44:01.309250: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Predicting: |                                             | 0/? [00:00<?, ?it/s]

Epoch: 550 | Inputs: 10 | MLP units: [[250, 250], [250, 250]] | Scaler Type: robust
Pooling mode: AvgPool1d | Learning Rate: 0 | Block: [1, 1]| Dropout:0  
RMSE: 0.009927744107958361 | QLIKE: 0.13157798478993074 | MAPE: 0.0066970986949069404


In [15]:
for m in range (4):
  i=random.randrange(7)
  h=random.randrange(4)
  a=random.randrange(4)
  k=random.randrange(9)
  l=random.randrange(4)
  n=random.randrange(4)
  o=random.randrange(3)
  p=random.randrange(2)
  model = NHITS(h=28, input_size=n_inputs[i],
                loss=MSE(),
                scaler_type=scaler_type[o],
                learning_rate=learning_rate[n],
                stack_types=['identity','identity'],
                n_blocks=n_blocks[a],
                mlp_units=mlp_units[h],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[n],
                val_check_steps=500,
                stat_exog_list=['BTC'],
                dropout_prob_theta=dropout_prob_theta[l], n_pool_kernel_size=n_pool_kernel_size[o],
                pooling_mode = pooling_mode[p],
                interpolation_mode= interpolation_mode[p],
                max_steps=epochs[k],
                random_seed=random.randrange(129228148))
  fcst = NeuralForecast(models=[model],freq='D')
  forecasts = fcst.cross_validation(df=train,val_size=28,static_df=static_df,n_windows=None, test_size=validation_length-validation_length%28,step_size=1)
  forecasts = forecasts[forecasts['unique_id']=='ETH'].dropna().drop_duplicates(subset='ds', keep='first')
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHTS-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | MLP units: {mlp_units[h]} | Scaler Type: {scaler_type[o]}")
  print(f"Pooling mode: {pooling_mode[p]} | Learning Rate: {n} | Block: {n_blocks[a]}| Dropout:{dropout_prob_theta[l]}  ")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAPE: {MAE}")

Seed set to 107086890


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Predicting: |                                             | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Predicting: |                                             | 0/? [00:00<?, ?it/s]

Epoch: 650 | Inputs: 5 | MLP units: [[712, 712], [712, 712]] | Scaler Type: minmax
Pooling mode: MaxPool1d | Learning Rate: 3 | Block: [3, 3]| Dropout:0.4  
RMSE: 0.010908880605092297 | QLIKE: 0.14057066624544293 | MAPE: 0.007296628449751879


In [16]:
#first
for m in range (5):
  i=random.randrange(7)
  h=random.randrange(4)
  a=random.randrange(4)
  k=random.randrange(9)
  l=random.randrange(4)
  n=random.randrange(4)
  o=random.randrange(3)
  p=random.randrange(2)
  model = NHITS(h=28, input_size=n_inputs[i],
                loss=MSE(),
                scaler_type=scaler_type[o],
                learning_rate=learning_rate[n],
                stack_types=['identity','identity'],
                n_blocks=n_blocks[a],
                mlp_units=mlp_units[h],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[n],
                val_check_steps=500,
                stat_exog_list=['BTC'],
                dropout_prob_theta=dropout_prob_theta[l], n_pool_kernel_size=n_pool_kernel_size[o],
                pooling_mode = pooling_mode[p],
                interpolation_mode= interpolation_mode[p],
                max_steps=epochs[k],
                random_seed=random.randrange(129228148))
  fcst = NeuralForecast(models=[model],freq='D')
  forecasts = fcst.cross_validation(df=train,val_size=28,static_df=static_df,n_windows=None, test_size=validation_length-validation_length%28,step_size=1)
  forecasts = forecasts[forecasts['unique_id']=='ETH'].dropna().drop_duplicates(subset='ds', keep='first')
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHTS-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | MLP units: {mlp_units[h]} | Scaler Type: {scaler_type[o]}")
  print(f"Pooling mode: {pooling_mode[p]} | Learning Rate: {n} | Block: {n_blocks[a]}| Dropout:{dropout_prob_theta[l]}  ")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAPE: {MAE}")

Seed set to 71715799


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Predicting: |                                             | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Predicting: |                                             | 0/? [00:00<?, ?it/s]

Epoch: 350 | Inputs: 5 | MLP units: [[512, 512], [512, 512]] | Scaler Type: robust
Pooling mode: MaxPool1d | Learning Rate: 3 | Block: [5, 5]| Dropout:0.4  
RMSE: 0.010744549222048051 | QLIKE: 0.14129466828150036 | MAPE: 0.006695536540951884


In [17]:
#second
for m in range (5):
  i=random.randrange(7)
  h=random.randrange(4)
  a=random.randrange(4)
  k=random.randrange(9)
  l=random.randrange(4)
  n=random.randrange(4)
  o=random.randrange(3)
  p=random.randrange(2)
  model = NHITS(h=28, input_size=n_inputs[i],
                loss=MSE(),
                scaler_type=scaler_type[o],
                learning_rate=learning_rate[n],
                stack_types=['identity','identity'],
                n_blocks=n_blocks[a],
                mlp_units=mlp_units[h],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[n],
                val_check_steps=500,
                stat_exog_list=['BTC'],
                dropout_prob_theta=dropout_prob_theta[l], n_pool_kernel_size=n_pool_kernel_size[o],
                pooling_mode = pooling_mode[p],
                interpolation_mode= interpolation_mode[p],
                max_steps=epochs[k],
                random_seed=random.randrange(129228148))
  fcst = NeuralForecast(models=[model],freq='D')
  forecasts = fcst.cross_validation(df=train,val_size=28,static_df=static_df,n_windows=None, test_size=validation_length-validation_length%28,step_size=1)
  forecasts = forecasts[forecasts['unique_id']=='ETH'].dropna().drop_duplicates(subset='ds', keep='first')
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHTS-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | MLP units: {mlp_units[h]} | Scaler Type: {scaler_type[o]}")
  print(f"Pooling mode: {pooling_mode[p]} | Learning Rate: {n} | Block: {n_blocks[a]}| Dropout:{dropout_prob_theta[l]}  ")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAPE: {MAE}")

Seed set to 96693490


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Predicting: |                                             | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Predicting: |                                             | 0/? [00:00<?, ?it/s]

Epoch: 100 | Inputs: 3 | MLP units: [[512, 512], [512, 512]] | Scaler Type: standard
Pooling mode: AvgPool1d | Learning Rate: 2 | Block: [3, 3]| Dropout:0  
RMSE: 0.013411413604735573 | QLIKE: 0.15913676797830156 | MAPE: 0.008244336777377029


In [18]:
#third
for m in range (5):
  i=random.randrange(7)
  h=random.randrange(4)
  a=random.randrange(4)
  k=random.randrange(9)
  l=random.randrange(4)
  n=random.randrange(4)
  o=random.randrange(3)
  p=random.randrange(2)
  model = NHITS(h=28, input_size=n_inputs[i],
                loss=MSE(),
                scaler_type=scaler_type[o],
                learning_rate=learning_rate[n],
                stack_types=['identity','identity'],
                n_blocks=n_blocks[a],
                mlp_units=mlp_units[h],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[n],
                val_check_steps=500,
                stat_exog_list=['BTC'],
                dropout_prob_theta=dropout_prob_theta[l], n_pool_kernel_size=n_pool_kernel_size[o],
                pooling_mode = pooling_mode[p],
                interpolation_mode= interpolation_mode[p],
                max_steps=epochs[k],
                random_seed=random.randrange(129228148))
  fcst = NeuralForecast(models=[model],freq='D')
  forecasts = fcst.cross_validation(df=train,val_size=28,static_df=static_df,n_windows=None, test_size=validation_length-validation_length%28,step_size=1)
  forecasts = forecasts[forecasts['unique_id']=='ETH'].dropna().drop_duplicates(subset='ds', keep='first')
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHTS-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | MLP units: {mlp_units[h]} | Scaler Type: {scaler_type[o]}")
  print(f"Pooling mode: {pooling_mode[p]} | Learning Rate: {n} | Block: {n_blocks[a]}| Dropout:{dropout_prob_theta[l]}  ")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAPE: {MAE}")

Seed set to 71661734


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Predicting: |                                             | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Predicting: |                                             | 0/? [00:00<?, ?it/s]

Epoch: 650 | Inputs: 3 | MLP units: [[250, 250], [250, 250]] | Scaler Type: minmax
Pooling mode: AvgPool1d | Learning Rate: 3 | Block: [5, 5]| Dropout:0.4  
RMSE: 0.014791502908009535 | QLIKE: 0.1709365966952638 | MAPE: 0.00895599716845434


In [19]:
#fourth
for m in range (5):
  i=random.randrange(7)
  h=random.randrange(4)
  a=random.randrange(4)
  k=random.randrange(9)
  l=random.randrange(4)
  n=random.randrange(4)
  o=random.randrange(3)
  p=random.randrange(2)
  model = NHITS(h=28, input_size=n_inputs[i],
                loss=MSE(),
                scaler_type=scaler_type[o],
                learning_rate=learning_rate[n],
                stack_types=['identity','identity'],
                n_blocks=n_blocks[a],
                mlp_units=mlp_units[h],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[n],
                val_check_steps=500,
                stat_exog_list=['BTC'],
                dropout_prob_theta=dropout_prob_theta[l], n_pool_kernel_size=n_pool_kernel_size[o],
                pooling_mode = pooling_mode[p],
                interpolation_mode= interpolation_mode[p],
                max_steps=epochs[k],
                random_seed=random.randrange(129228148))
  fcst = NeuralForecast(models=[model],freq='D')
  forecasts = fcst.cross_validation(df=train,val_size=28,static_df=static_df,n_windows=None, test_size=validation_length-validation_length%28,step_size=1)
  forecasts = forecasts[forecasts['unique_id']=='ETH'].dropna().drop_duplicates(subset='ds', keep='first')
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHTS-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | MLP units: {mlp_units[h]} | Scaler Type: {scaler_type[o]}")
  print(f"Pooling mode: {pooling_mode[p]} | Learning Rate: {n} | Block: {n_blocks[a]}| Dropout:{dropout_prob_theta[l]}  ")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAPE: {MAE}")

Seed set to 84453367


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Predicting: |                                             | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Predicting: |                                             | 0/? [00:00<?, ?it/s]

Epoch: 650 | Inputs: 84 | MLP units: [[250, 250], [250, 250]] | Scaler Type: robust
Pooling mode: MaxPool1d | Learning Rate: 0 | Block: [5, 5]| Dropout:0  
RMSE: 0.008011690748731459 | QLIKE: 0.169762378989087 | MAPE: 0.005561464579958127


In [20]:
#fifth
for m in range (5):
  i=random.randrange(7)
  h=random.randrange(4)
  a=random.randrange(4)
  k=random.randrange(9)
  l=random.randrange(4)
  n=random.randrange(4)
  o=random.randrange(3)
  p=random.randrange(2)
  model = NHITS(h=28, input_size=n_inputs[i],
                loss=MSE(),
                scaler_type=scaler_type[o],
                learning_rate=learning_rate[n],
                stack_types=['identity','identity'],
                n_blocks=n_blocks[a],
                mlp_units=mlp_units[h],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[n],
                val_check_steps=500,
                stat_exog_list=['BTC'],
                dropout_prob_theta=dropout_prob_theta[l], n_pool_kernel_size=n_pool_kernel_size[o],
                pooling_mode = pooling_mode[p],
                interpolation_mode= interpolation_mode[p],
                max_steps=epochs[k],
                random_seed=random.randrange(129228148))
  fcst = NeuralForecast(models=[model],freq='D')
  forecasts = fcst.cross_validation(df=train,val_size=28,static_df=static_df,n_windows=None, test_size=validation_length-validation_length%28,step_size=1)
  forecasts = forecasts[forecasts['unique_id']=='ETH'].dropna().drop_duplicates(subset='ds', keep='first')
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHTS-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | MLP units: {mlp_units[h]} | Scaler Type: {scaler_type[o]}")
  print(f"Pooling mode: {pooling_mode[p]} | Learning Rate: {n} | Block: {n_blocks[a]}| Dropout:{dropout_prob_theta[l]}  ")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAPE: {MAE}")

Seed set to 49928015


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Predicting: |                                             | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Predicting: |                                             | 0/? [00:00<?, ?it/s]

Epoch: 450 | Inputs: 21 | MLP units: [[512, 512], [512, 512]] | Scaler Type: standard
Pooling mode: MaxPool1d | Learning Rate: 2 | Block: [3, 3]| Dropout:0.2  
RMSE: 0.008831174127707831 | QLIKE: 0.1174238895107549 | MAPE: 0.006072026715535876


In [ ]:
#Hyperparameter search

n_inputs =[3,5,10,15,21,42,84]
mlp_units = [[[712, 712], [712, 712]],[[512, 512], [512, 512]],[[250, 250], [250, 250]],[[100, 100], [100, 100]]]
epochs=[50,100,150,250,350,450,550,650,750]
learning_rate=[0.0005,0.0001,0.00005,0.00001]
num_lr_decays=[5,3,2,1]
scaler_type=["robust","standard",'minmax']
dropout_prob_theta=[0, 0.2, 0.3, 0.4]
n_pool_kernel_size=[[2,2,1], [4,2,1], [4,4,2]]
pooling_mode=['MaxPool1d', 'AvgPool1d']
interpolation_mode=['linear', 'nearest']
n_blocks=[[1, 1],[2, 2],[3, 3],[5, 5]]


In [ ]:
Seed set to 25517486
Epoch: 650 | Inputs: 84 | MLP units: [[250, 250], [250, 250]] | Scaler Type: robust
Pooling mode: MaxPool1d | Learning Rate: 0 | Block: [5, 5]| Dropout:0  
RMSE: 0.008011690748731459 | QLIKE: 0.169762378989087 | MAPE: 0.005561464579958127

In [21]:
#random seed tuning
for m in range (5):
  randomseed=random.randrange(25517486)
  model = NHITS(h=28, input_size=84,
                loss=MSE(),
                scaler_type='robust',
                learning_rate=learning_rate[0],
                stack_types=['identity','identity'],
                n_blocks=[5,5],
                mlp_units=[[250, 250], [250, 250]],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[0],
                val_check_steps=500,
                stat_exog_list=['BTC'],
                dropout_prob_theta=0, n_pool_kernel_size=[2,2,1],
                pooling_mode = 'MaxPool1d',
                interpolation_mode= 'linear',
                max_steps=650,
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq='D')
  forecasts = fcst.cross_validation(df=train,val_size=28,static_df=static_df,n_windows=None, test_size=validation_length-validation_length%28,step_size=1)
  forecasts = forecasts[forecasts['unique_id']=='ETH'].dropna().drop_duplicates(subset='ds', keep='first')
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHTS-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAPE: {MAE}")

Seed set to 19161429


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Predicting: |                                             | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Predicting: |                                             | 0/? [00:00<?, ?it/s]

Random Seed: 16606075
RMSE: 0.007881513071007006 | QLIKE: 0.1604941963136862 | MAPE: 0.005393948455261742


In [22]:
#second
for m in range (5):
  randomseed=random.randrange(25517486)
  model = NHITS(h=28, input_size=84,
                loss=MSE(),
                scaler_type='robust',
                learning_rate=learning_rate[0],
                stack_types=['identity','identity'],
                n_blocks=[5,5],
                mlp_units=[[250, 250], [250, 250]],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[0],
                val_check_steps=500,
                stat_exog_list=['BTC'],
                dropout_prob_theta=0, n_pool_kernel_size=[2,2,1],
                pooling_mode = 'MaxPool1d',
                interpolation_mode= 'linear',
                max_steps=650,
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq='D')
  forecasts = fcst.cross_validation(df=train,val_size=28,static_df=static_df,n_windows=None, test_size=validation_length-validation_length%28,step_size=1)
  forecasts = forecasts[forecasts['unique_id']=='ETH'].dropna().drop_duplicates(subset='ds', keep='first')
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHTS-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAPE: {MAE}")

Seed set to 16288969


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Predicting: |                                             | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Predicting: |                                             | 0/? [00:00<?, ?it/s]

Random Seed: 4498536
RMSE: 0.007990816741122076 | QLIKE: 0.18984878332575228 | MAPE: 0.0053182064251797575


In [23]:
#third
for m in range (5):
  randomseed=random.randrange(25517486)
  model = NHITS(h=28, input_size=84,
                loss=MSE(),
                scaler_type='robust',
                learning_rate=learning_rate[0],
                stack_types=['identity','identity'],
                n_blocks=[5,5],
                mlp_units=[[250, 250], [250, 250]],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[0],
                val_check_steps=500,
                stat_exog_list=['BTC'],
                dropout_prob_theta=0, n_pool_kernel_size=[2,2,1],
                pooling_mode = 'MaxPool1d',
                interpolation_mode= 'linear',
                max_steps=650,
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq='D')
  forecasts = fcst.cross_validation(df=train,val_size=28,static_df=static_df,n_windows=None, test_size=validation_length-validation_length%28,step_size=1)
  forecasts = forecasts[forecasts['unique_id']=='ETH'].dropna().drop_duplicates(subset='ds', keep='first')
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHTS-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAPE: {MAE}")

Seed set to 14550156


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Predicting: |                                             | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Predicting: |                                             | 0/? [00:00<?, ?it/s]

Random Seed: 20710803
RMSE: 0.008250472480717535 | QLIKE: 0.22233881179500856 | MAPE: 0.005776082558702273


In [24]:
#Testing
test_length=int(len(tabData[tabData['unique_id']=='BTC'])*0.3)

In [25]:
model = NHITS(h=28, input_size=84,
                loss=MSE(),
                scaler_type='robust',
                learning_rate=learning_rate[0],
                stack_types=['identity','identity'],
                n_blocks=[5,5],
                mlp_units=[[250, 250], [250, 250]],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[0],
                val_check_steps=500,
                stat_exog_list=['BTC'],
                dropout_prob_theta=0, n_pool_kernel_size=[2,2,1],
                pooling_mode = 'MaxPool1d',
                interpolation_mode= 'linear',
                max_steps=650,
                random_seed=16606075)
fcst = NeuralForecast(models=[model],freq='D')
forecasts = fcst.cross_validation(df=tabData,val_size=28,static_df=static_df,n_windows=None, test_size=test_length-test_length%28,step_size=1)
forecasts = forecasts[forecasts['unique_id']=='ETH'].dropna().drop_duplicates(subset='ds', keep='first')
if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
else:
    Y_hat=forecasts["NHTS-median"].values
Y_true=forecasts["y"].values
RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)

print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAPE: {MAE}")

Seed set to 16606075


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Predicting: |                                             | 0/? [00:00<?, ?it/s]

RMSE: 0.003617969651877506 | QLIKE: 0.1379416905301119 | MAPE: 0.0027837998483489015


In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np
fig = px.line(forecasts, x='ds', y=forecasts.columns[3:], color='variable', title='H=28 N-HITS prediction')

fig.show()

In [ ]:
forecasts=forecasts.rename(columns={'y':'Real_value'})
forecasts.to_csv('ETH-H=28-NHITS.csv')